# Lesson 5 - Errors and Troubleshooting STATUS-- NOT STARTED

## Introduction

In this lesson we would try to provide a list of possible errors that you might run into and provide information on how to address those issues as they arise. Also note that some warnings might seem like an error but are actually warnings or system glitches. We break down the errors to 4 sections, depending on the procedure that they might occur. 
* Initialization 
* SpinUP
* Calibration 
* Validation 

## Initialization 

In [1]:
#_______DEV ARCHIVE_________


## Introduction 

In this lesson we would try to provide a list of possible erorrs that you might run into and provide information on how to address those issues if you faced it. Also some warning that might seem like an error but they are not errors in fact, and are warnings or system glitches. We break down the errors to 4 sections, depending on the procedure that they might occure. 
* Initialization 
* SpinUP
* Calibration 
* Validation 

## Initialization 

There are few things that could cause error in the initialization steps (covered in Lesson 2), few of them were mentioned in Lesson 2 and we will review them here also. 

# PATRICK: I am going to number the issues and leave you title ... Please try to explain it or recreate the error if possible, 
# For the ones that have editing, let s not count on opening vi, and just use the ptyhon/bash commands to edit the file. If you need example, there should be in the old training lessons, where we changed namelist options. If you do not know how, ask me and I will look for it. 

Issue 1: (Hands on) What if the database exists? --- hands on, Patrick add a cell where you call the initDB and try to create a database that already exist (from Lesson 2) .... let try to overwrite to example1

For 2 and 3, lets create a copy of the domainMeta.csv and make the edits in there to avoid losing the right file. 
Issue 2: (Hands on) What if the doaminMeta does not have a column name right, for example change the name of the column gageId and see what happens. (exmaple2) 

Issue 3; (Hands on) What if the domainMeta is missing an required values, for example the value for gageId is missing or another attribute. 


For the following issues, if it ivolves editing the setup.parm file, let s make a copy of the file and edit that. 
Issue 4: What if the run directory specified in the setup file does not exist? (smaple: *ERROR: Failure to create directory: /glade/scratch/arezoo/RFC_Training2/01447680*) 

Issue 5: What if the run directory was created in in a previous trial (Sample: ERROR: Top level directory: /glade/u/home/arezoo/scratch/NWM_V30_CALIB_SOIL/NC_Soil_Regular already exists
) 

Issue 6: (HAnds on), what if one of the options in the setup.parm file does not exist? Let s for example comment out the line for enableMask = 0, and see what happens. 

Issue 7: (NOT PRIORITY: Hands on), what if there are more options than reqried, lets add something to the setup file. I am not sure if this throws any erro though... 

Issue 8: (Hands on) What if the path to one of the necessary files does not exist like the wrfHydroexe file or any of the TBL files. Even if they are not required, they should be there execpt from the urbn and veg ones that I edited the code right now to handle that condition. 

Issue 9:(Hands on) What if we try to enter job IDs in the database (same numbers)? (sample; ERROR: Information for this job has already been entered as job ID: 100
)

#### Subheader:
Text


# SPINUP
Few things could cause error in the SPINUP (covered in Lesson 3). Most of the errors could be caught by the calibration workflow and if the email address is provided in the `setup.parm`  file, then a proper message will be sent out to the user and specifying that there is a problem and user needs to address it. Usually there are two types of errors that could happen at this stage. Depending on which one crashes, a LOCK file is created.

*  WRF-Hydro model crashes. If the model crashes, the file will be called `RUN.LOCK` and it will be placed under the `RUN.SPINIP` directory.
*  Calibration workflow (R scripts) crashes. If the calibration workflow crashes, then a LOCK file called `CALIB.LOCK` will be created and is placed under the `RUN.SPINUP` directory. 

The workflow will be running in the background but since there is LOCK file, it will not do anything and will hold still until you fix the issue and remove the LOCK file. After removing the LOCK file, the procedure will restart and pick up from it crashed.

## How the content of the email looks like?
* If WRF-Hydro model fails, it would be a message like: `ERROR: SIMULATION FOR GAGE: 01447720 HAS FAILED A SECOND TIME. PLEASE FIX ISSUE AND MANUALLY REMOVE LOCK FILE: /home/docker/example_case/Calibration/output/example1/01447720/RUN.SPINUP/RUN.LOCK`
* If calibration workflow fails, user would receive a messgae like the following: `ERROR: Calibration Scripts failed a second time for gage: 01447720 ... Please remove LOCKFILE: /home/docker/example_case/Calibration/output/example1/01447720/RUN.SPINUP/CALIB.LOCK`

## Where to search for error messages?
* if WRF-Hydro model fails, you need to search for the standard error and out in the `RUN.SPINUP/OUTPUT` directory. The standard error and out files would be called something like: `/home/docker/example_case/Calibration/output/example1/01447720/RUN.SPINUP/OUTPUT/WH_1_1.err and WH_1_1.out`. These two files are the first place to look for hints of why the model failed. You could also look at the diag files in the same folder, sometimes those files have better info in them. 
* if the calibration workflow fails, you will see a standart error and out file in the `RUN.SPINUP/OUTPUT` directory which look like this: `/home/docker/example_case/Calibration/output/example1/01447720/RUN.SPINUP/OUTPUT/WH_CALIB_1_1.err and WH_CALIB_1_1.out`. The error information should be found there. 


## Errors not caught by workflow: 
Some of the errors do not get caught by the workflow and those are the most difficult ones to figure out. Below is a few of the possible failures that could happen. 
### Not finding restart files at the end of spin up date. 
User specify the start and end date of the spip up period in the `setup.parm` file. User also specified the frequency of restart files (for both LSM and hydro) in the `setup.parm` file. The workflow looks for the restart file at the end date requested for the spin up as an indication tha the model run is complete. So if the user define the sin up period and the restart frequency in a way that there will not be any restart files outputted at the end of the simulations, python workflow does not see the restart file and will fall into an infite loop. Our common practice is to define the perios (both start/end) for spin up, calibration and validation at the start of the month and set the frequency of restart files to -9999 which will output restart files once every month. 

### Fixed Grid ID: 
Grid ID is hard-coded to 1 in the calibration workflow, meaning it is expecting the restart files to be named like `???DOMAIN1` and if they are not, the model does not see the resart files, and think that the job has not finished, and restart it. In this case, the workflow will be in an inifite loop and does not finish. If your domain is an NWM cutout, then this problem will not arise. However, if you are using a non NWM domain, make sure the `grid_id` global attribute in the `wrfinput.nc` file is 1. This is where the LSM will grab the index of the domain. 

## Workflow needs both restart files:
User cannot turn off all the routings, since the workflow look for both restart files (LSM and hydro) as an indication of job completion. If user turns off all the routing options, there will not be any hydro restart file, and therefore the model will fall into in infinite loop. 


## Calibration 





## Validation 

What is the valid_workflow fails?  CALIB.LOCK 
What if the model fails .. where is the RUN.LOCK file 
How to run valid_workflow to diagnose?


## Random Warnings: 

Sometimes there is system glitches and the workflow cannot update a field in the database, they you will get email message like these below: 
* ERROR: Unable to query domain meta table for gages metadata.
* ERROR: Unable to update calibration status for job ID: 4 domainID: 28 Iteration: 430
* ERROR: Failure to enter value for parameter: bexp jobID: 4 domainID: 40 iteration: 451

These are not really erorrs and will be resolved by the python workflow and can be ignored. 


## Bug that will be addressed:
As explained there are a number of directories created under `RUN.CALIB` directory, one of them is called `FINAL_PARAMETERS`. Unfortunatele, this dir does not have the files associated to the best iteraiton. It contains the files from the next iteration of the best one. We are aware of the bug, and will address it soon. 

Meanwhile, if user needs the domain files with the best parameters, you couild find them under `RUN.VALID/OUTPUT/BEST` directory. 

## Conclusion:

we just went through a few of the errors that user could encounter and how to address them. This list is not complete, please contact us if you run into any issue that is not explained here. 